<a href="https://colab.research.google.com/github/Anuradha04/python_projects/blob/main/Email_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

asks for parameters,
summerize using transformers
To read gmail: use simple gmail

**Steps to follow: **
0. google developer console -  create project - api overview - api lib- gmail api - enable - oAuth console - test users add users in it.
1. credientials - create credentials - oAuth - download json file- rename as "client secret"(make sure its in json).
2. at block 5(upload) , run - choose file- "client-secret" - then rerun until you get verification block - go to the link, allow all - copy the authenication code and paste it in the verification block, then enter.
3.  Summary - is a pipeline process = has many models, can find different model from huggingface, inside summary under quotations



In [ ]:
!pip install --upgrade google-api-python-client oauth2client simplegmail
!pip -q install txtai[pipeline] > /dev/null

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 26.4 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.137.0
    Uninstalling google-api-python-client-2.137.0:
      Successfully uninstalled google-api-python-client-2.137.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 3.20.2 which is incompatible.


In [ ]:
!pip install --force-reinstall protobuf==3.20.3

  Using cached protobuf-3.20.3-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (679 bytes)
Using cached protobuf-3.20.3-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
onnxconverter-common 1.14.0 requires protobuf==3.20.2, but you have protobuf 3.20.3 which is incompatible.


In [ ]:
from google.colab import files
from simplegmail import Gmail
from simplegmail.query import construct_query
from txtai.pipeline import Summary

In [ ]:

# Upload the client_secret.json file to Colab
uploaded = files.upload()  # run and then Use this to upload the 'client_secret.json' file

# Initialize Gmail object
gmail = Gmail()  # It will open a link for OAuth authentication


Saving client_secret.json to client_secret (1).json

Failed to start a local webserver listening on either port 8080
or port 8090. Please check your firewall settings and locally
running programs that may be blocking or using those ports.

Falling back to --noauth_local_webserver and continuing with
authorization.


Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=61126731814-dhj0pbsk0350h5si24fdjo55nd9shjj2.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.settings.basic&access_type=offline&response_type=code&prompt=consent

Enter verification code: 4/1AQlEd8yYQ2prdDbwEyjpGNkUCu_Q2OipNzhnxFr1DZyxxKbDG4I2DnMi4So
Authentication successful.


In [ ]:
"""params={
    #'read'=True, # also can be false - both read and unread mails will be read
    'senders'=['noreply@medium.com',]

}"""

In [ ]:
#setting parameters - filters
def get_query_params():
    query_params = {}

    # Get 'read' status
    while True:
        read_input = input("Enter 'read' status (True/False) or press Enter to skip: ")
        if read_input == '':
            break  # Skip this field if user presses Enter
        elif read_input.lower() in ['true', 'false']:
            query_params["read"] = read_input.lower() == 'true'
            break
        else:
            print("Invalid input. Please enter 'True' or 'False'.")

    # Get 'recipient'- new
    while True:
        recipient_input = input("Enter recipient email(s) (comma-separated) or press Enter to skip: ")
        if recipient_input == '':
            break
        elif all('@' in email.strip() for email in recipient_input.split(',')):
            query_params["recipient"] = [email.strip() for email in recipient_input.split(',')]
            break
        else:
            print("Invalid input. Please enter valid email addresses.")


    # Get 'sender'
    while True:
        sender_input = input("Enter sender email(s) (comma-separated) or press Enter to skip: ")
        if sender_input == '':
            break  # Skip this field if user presses Enter
        elif all('@' in email.strip() for email in sender_input.split(',')):
            query_params["sender"] = [email.strip() for email in sender_input.split(',')]
            break
        else:
            print("Invalid input. Please enter valid email addresses.")

    # Get 'newer_than'
    while True:
        newer_than_input = input("Enter 'newer_than' duration (e.g., '5 day') or press Enter to skip: ")
        if newer_than_input == '':
            break  # Skip this field if user presses Enter
        try:
            number, unit = newer_than_input.split()
            query_params["newer_than"] = (int(number), unit)
            break
        except ValueError:
            print("Invalid input for 'newer_than'. It should be in the format: '<number> <unit>'")

    # Get 'labels'
    while True:
        labels_input = input("Enter labels (comma-separated, e.g., 'CATEGORY_FORUMS, INBOX') or press Enter to skip: ")
        if labels_input == '':
            break  # Skip this field if user presses Enter
        elif all(label.strip() for label in labels_input.split(',')):
            query_params["labels"] = [[label.strip()] for label in labels_input.split(',')]
            break
        else:
            print("Invalid input. Please enter valid labels.")
    # Get 'subject'
    while True:
        subject_input = input("Enter subject keyword(s) (comma-separated) or press Enter to skip: ")
        if subject_input == '':
            break
        query_params["subject"] = subject_input.split(',')
        break


    return query_params

# Call the function to get user input and print the resulting query_params
query_params = get_query_params()
print("\nGenerated query_params:")
print(query_params)


Enter 'read' status (True/False) or press Enter to skip: 
Enter recipient email(s) (comma-separated) or press Enter to skip: 
Enter sender email(s) (comma-separated) or press Enter to skip: 
Enter 'newer_than' duration (e.g., '5 day') or press Enter to skip: 2 day
Enter labels (comma-separated, e.g., 'CATEGORY_FORUMS, INBOX') or press Enter to skip: 
Enter subject keyword(s) (comma-separated) or press Enter to skip: software

Generated query_params:
{'newer_than': (2, 'day'), 'subject': ['software']}


In [ ]:
promo_query = construct_query(query_params)

output = gmail.get_messages(query=promo_query)

In [ ]:
# Create and run pipeline
summary = Summary()

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
html_content = """
<html>
<head>
  <style>
    @import url('https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&display=swap');
    @import url('https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.4/css/all.min.css');

    body {
      font-family: 'Roboto', sans-serif;
      background-color: #f0f4f8;
      margin: 0;
      padding: 0;
      color: #333;
    }

    .email-summary {
      width: 100%;
      max-width: 600px;
      margin: 20px auto;
      padding: 20px;
      background-color: #ffffff;
      border-radius: 10px;
      box-shadow: 0 4px 8px rgba(0,0,0,0.1);
    }

    .heading {
      text-align: center;
      background-color: #ff4d4d;  /* Red background for the header */
      padding: 15px;
      border-radius: 10px;
      color: #fff;  /* White text */
    }

    .subheading {
      text-align: center;
      font-size: 18px;
      color: #333;
      margin: 20px 0;
      font-weight: 700;
    }

    .email-item {
      background-color: #f9fafc;
      padding: 20px;
      margin: 10px 0;
      border-radius: 8px;
      border: 1px solid #e0e6ed;
    }

    .email-header {
      font-weight: 700;
      color: #333;  /* Black for headers */
      display: flex;
      align-items: center;
    }

    .email-header i {
      margin-right: 10px;
      color: #ff4d4d;  /* Red icons */
    }

    .email-body {
      margin-top: 15px;
      font-size: 14px;
      color: #555;
      line-height: 1.6;
    }

    .email-footer {
      font-size: 12px;
      text-align: center;
      margin-top: 30px;
      color: #999;
    }

    .email-footer i {
      color: #ff4d4d;  /* Red for footer icons */
    }

    .subject {
      color: #ff4d4d;  /* Red for subject */
      font-size: 16px;
      font-weight: bold;
    }

    .date {
      font-size: 14px;
      color: #6c757d;
    }

    .btn {
      background-color: #ff4d4d;  /* Red button */
      color: white;
      padding: 10px 15px;
      border: none;
      border-radius: 5px;
      text-align: center;
      text-decoration: none;
      display: inline-block;
      font-size: 14px;
      margin-top: 20px;
      cursor: pointer;
    }

    .btn:hover {
      background-color: #e63939;  /* Darker red on hover */
    }
  </style>
</head>
<body>
  <div class="email-summary">
    <!-- Heading Section -->
    <div class="heading">
      <h1>Email Summarizer</h1>
    </div>

    <!-- Subheading Section -->
    <div class="subheading">
      Here's your email summary
    </div>
"""


In [ ]:
# Assuming output is a list of mail objects
mail_contents=""
for i, mail in enumerate(output):  # Use enumerate to get an index
  html_content += f"""
  <div class="email-item">
    <div class="email-header">
      <i class="fas fa-user-circle"></i> Sender: {mail.sender}
    </div>
    <div class="email-header subject">
      <i class="fas fa-envelope"></i> Subject: {mail.subject}
    </div>
    <div class="email-header date">
      <i class="fas fa-calendar-alt"></i> Date: {mail.date}
    </div>
    <div class="email-body">
  """
  # Writing plain text part of mail
  if mail.plain:
      summary_text = summary(mail.plain)
      html_content += summary_text
  else:
      html_content += "No plain text inside emails"

  html_content += "</div></div>\n"


In [ ]:
email = input("Enter your email address:\n")
params = {
    "to": email,
    "sender": "anuradha.navin04@gmail.com",
    "subject": "A summary for your inbox",
    "msg_plain": mail_contents,
    "msg_html": html_content
}

# Send the email
message = gmail.send_message(**params)

print('Email sent successfully!')


Enter your email address:
anuradha.navin04@gmail.com
Email sent successfully!
